In [16]:
from pathlib import Path
import sys

parent_dir = str(Path().resolve().parents[0])
sys.path.insert(0, parent_dir)

from app.utils import ShowMetadataList, ShowMetadata, metadata_to_dict

data = ShowMetadataList(
    shows=[
        ShowMetadata(
            "rotl",
            Path("../files/meta/rotl_dates.txt"),
            Path("../files/meta/rotl_titles.txt"),
        ),
        ShowMetadata(
            "roadwork",
            Path("../files/meta/roadwork_dates.txt"),
            Path("../files/meta/roadwork_titles.txt"),
        ),
    ]
)

dates_titles = metadata_to_dict(data)

In [17]:
from pathlib import Path

root_dir = Path("../files/summaries")


def episode_metadata(file_path):
    metadata = open(file_path, "r").read().split("\n\n")
    names, places, music, books, summary = metadata
    return {
        "names": names,
        "places": places,
        "music": music,
        "books": books,
        "summary": summary,
    }


out_dir = Path("../files/indexes")
out_dir.mkdir(parents=True, exist_ok=True)

files = [file for file in root_dir.rglob("*") if file.is_file()]

names = []
places = []
music = []
books = []
summaries = []

for file in files:
    showname = str(Path(file.relative_to(root_dir)).parent)
    episode = file.stem.split("_-_")[0]
    date = dates_titles[showname]["dates"][episode]
    title = dates_titles[showname]["titles"][episode]
    full_showname = ""
    if showname == "rotl":
        full_showname = "Roderick on the Line"
    if showname == "roadwork":
        full_showname = "Roadwork"
    metadata = episode_metadata(file)
    names.append(f"{full_showname} -Episode {episode} - {title}\n\n{metadata["names"]}")
    places.append(
        f"{full_showname} -Episode {episode} - {title}\n\n{metadata["places"]}"
    )
    music.append(f"{full_showname} -Episode {episode} - {title}\n\n{metadata["music"]}")
    books.append(f"{full_showname} -Episode {episode} - {title}\n\n{metadata["books"]}")
    summaries.append(
        f"{full_showname} -Episode {episode} - {title}\n\n{metadata["summary"]}"
    )

names_path = out_dir / "names.md"
places_path = out_dir / "places.md"
music_path = out_dir / "music.md"
books_path = out_dir / "books.md"
summaries_path = out_dir / "summaries.md"

with open(names_path, "w") as f:
    f.write("\n\n".join(names))
with open(places_path, "w") as f:
    f.write("\n\n".join(places))
with open(music_path, "w") as f:
    f.write("\n\n".join(music))
with open(books_path, "w") as f:
    f.write("\n\n".join(books))
with open(summaries_path, "w") as f:
    f.write("\n\n".join(summaries))